In [36]:
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

import json

In [37]:
# Use dask to read in the csv file as a dataframe
listings_df = dd.read_csv("Resources/listings.csv", assume_missing = True)

In [38]:
# Select only the columns we want for our analysis
listings_df = listings_df[["id", "neighbourhood", "latitude", "longitude", "room_type", "price", "availability_365", "calculated_host_listings_count"]].copy()
listings_df.head()

,id,neighbourhood,latitude,longitude,room_type,price,availability_365,calculated_host_listings_count
0,360.0,Highland,39.766415,-105.002098,Entire home/apt,91.0,179.0,2.0
1,364.0,Five Points,39.766720,-104.979060,Entire home/apt,179.0,358.0,1.0
2,590.0,North Park Hill,39.755110,-104.911090,Private room,59.0,146.0,2.0
3,592.0,North Park Hill,39.754810,-104.911060,Private room,58.0,210.0,2.0
4,1940.0,Baker,39.720205,-104.997499,Entire home/apt,77.0,243.0,1.0


In [39]:
# Find datatypes for each column
print(listings_df.dtypes)

id                                        float64
neighbourhood                     string[pyarrow]
latitude                                  float64
longitude                                 float64
room_type                         string[pyarrow]
price                                     float64
availability_365                          float64
calculated_host_listings_count            float64
dtype: object


In [40]:
# Change datatypes
listings_df["id"] = listings_df["id"].astype(int)
listings_df["availability_365"] = listings_df["availability_365"].astype(int)
listings_df["calculated_host_listings_count"] = listings_df["calculated_host_listings_count"].astype(int)

# Rename columns
listings_df = listings_df.rename(columns={
    "id": "ID",
    "neighbourhood": "Neighborhood",
    "latitude": "Latitude",
    "longitude": "Longitude",
    "room_type": "Listing Type",
    "price": "Price",
    "availability_365": "Available Days per Year",
    "calculated_host_listings_count": "Host Listings Count"
})

In [41]:
# Check datatypes were successfully changed
print(listings_df.dtypes)

ID                                   int64
Neighborhood               string[pyarrow]
Latitude                           float64
Longitude                          float64
Listing Type               string[pyarrow]
Price                              float64
Available Days per Year              int64
Host Listings Count                  int64
dtype: object


In [42]:
# Find out if there are null values in each column
print(listings_df.isnull().sum().compute())

ID                           0
Neighborhood                 0
Latitude                     0
Longitude                    0
Listing Type                 0
Price                      573
Available Days per Year      0
Host Listings Count          0
dtype: int64


In [43]:
# Remove all rows with null values
listings_df = listings_df.dropna()

In [44]:
# Recheck for null values
print(listings_df.isnull().sum().compute())

ID                         0
Neighborhood               0
Latitude                   0
Longitude                  0
Listing Type               0
Price                      0
Available Days per Year    0
Host Listings Count        0
dtype: int64


In [45]:
# Get statistical information
summary_stats = listings_df[["Latitude", "Longitude", "Price", "Available Days per Year"]].describe()
print(summary_stats.compute())

          Latitude    Longitude        Price  Available Days per Year
count  4578.000000  4578.000000  4578.000000              4578.000000
mean     39.742149  -104.975373   162.639144               219.004806
std       0.031772     0.061382   239.225490               116.491189
min      39.625750  -105.102804    10.000000                 0.000000
25%      39.728498  -105.016473    84.000000               116.000000
50%      39.748135  -104.984050   117.000000               237.000000
75%      39.762350  -104.957370   178.000000               334.000000
max      39.847417  -104.673817  9999.000000               365.000000


In [46]:
# Find out how many listings have "Available Days per Year" as 0
print(listings_df.query("`Available Days per Year` == 0").count().compute())

ID                         77
Neighborhood               77
Latitude                   77
Longitude                  77
Listing Type               77
Price                      77
Available Days per Year    77
Host Listings Count        77
dtype: int64


In [47]:
# Filter out all rows where "Available Days per Year" is 0
listings_df = listings_df.query("`Available Days per Year` ! = 0")

In [48]:
# Look at statistical information again after filtering out rows
summary_stats = listings_df[["Latitude", "Longitude", "Price", "Available Days per Year"]].describe()
print(summary_stats.compute())

          Latitude    Longitude        Price  Available Days per Year
count  4501.000000  4501.000000  4501.000000              4501.000000
mean     39.742191  -104.975155   162.798267               222.751389
std       0.031843     0.061594   240.860574               113.875617
min      39.625750  -105.102804    10.000000                 1.000000
25%      39.728490  -105.016480    84.000000               121.000000
50%      39.748120  -104.983920   116.000000               240.000000
75%      39.762540  -104.956978   178.000000               335.000000
max      39.847417  -104.673817  9999.000000               365.000000


In [49]:
listings_df_cleaned = listings_df.copy()
listings_df_cleaned.head()

,ID,Neighborhood,Latitude,Longitude,Listing Type,Price,Available Days per Year,Host Listings Count
0,360,Highland,39.766415,-105.002098,Entire home/apt,91.0,179,2
1,364,Five Points,39.766720,-104.979060,Entire home/apt,179.0,358,1
2,590,North Park Hill,39.755110,-104.911090,Private room,59.0,146,2
3,592,North Park Hill,39.754810,-104.911060,Private room,58.0,210,2
4,1940,Baker,39.720205,-104.997499,Entire home/apt,77.0,243,1


In [50]:
# Export cleaned data to a new csv
listings_df_cleaned.to_csv('Resources/cleaned_listings.csv', index=False, single_file=True)

['/Users/angelinamurdock/Desktop/Classwork/Project-5_Group-3/Resources/cleaned_listings.csv']

In [51]:
# Convert cleaned_listings.csv into a .json file 
listings_df = dd.read_csv('Resources/cleaned_listings.csv')

# Compute the DataFrame to Pandas (if the dataset fits into memory)
listings_pandas_df = listings_df.compute()

# Convert to list of dictionaries (this is how the data will be represented in JavaScript)
data_as_dict = listings_pandas_df.to_dict(orient='records')

# Write the JavaScript data to a .json file
with open("Resources/listings.json", "w") as json_file:
    json.dump(data_as_dict, json_file, indent=4)

In [62]:
# Use dask to read in  Neighborhood_zillow_real_estate_data.csv file as a dataframe
real_estate_neighborhood_df = dd.read_csv("Resources/Neighborhood_zillow_real_estate_data.csv", assume_missing = True)
real_estate_neighborhood_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28
0,112345.0,0.0,Maryvale,neighborhood,AZ,AZ,Phoenix,"Phoenix-Mesa-Chandler, AZ",Maricopa County,67583.518398,...,3.414316e+05,3.423949e+05,3.428074e+05,3.427356e+05,3.419887e+05,3.408405e+05,3.396659e+05,3.388768e+05,3.377961e+05,3.359831e+05
1,192689.0,1.0,Paradise,neighborhood,NV,NV,Las Vegas,"Las Vegas-Henderson-Paradise, NV",Clark County,131822.893122,...,3.811490e+05,3.835971e+05,3.857815e+05,3.874718e+05,3.887102e+05,3.896048e+05,3.907356e+05,3.922168e+05,3.934987e+05,3.939496e+05
2,270958.0,2.0,Upper West Side,neighborhood,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,381660.874369,...,1.169479e+06,1.169561e+06,1.173572e+06,1.182601e+06,1.187323e+06,1.191276e+06,1.203791e+06,1.222023e+06,1.237589e+06,1.245587e+06
3,270957.0,3.0,Upper East Side,neighborhood,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,624209.066407,...,1.161944e+06,1.160380e+06,1.162163e+06,1.168415e+06,1.171441e+06,1.173235e+06,1.184594e+06,1.201830e+06,1.216919e+06,1.222750e+06
4,118208.0,4.0,South Los Angeles,neighborhood,CA,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,130013.865244,...,6.633109e+05,6.635775e+05,6.661953e+05,6.714373e+05,6.780470e+05,6.834528e+05,6.871959e+05,6.899128e+05,6.883663e+05,6.834446e+05


In [63]:
# Select only the columns we want for our analysis
real_estate_neighborhood_df = real_estate_neighborhood_df[["RegionID", "RegionName", "State", "City", "2024-01-31", "2024-02-29", "2024-03-31", "2024-04-30", "2024-05-31", "2024-06-30", "2024-07-31", "2024-08-31", "2024-09-30", "2024-10-31", "2024-11-30", "2024-12-31"]].copy()
real_estate_neighborhood_df.head()

,RegionID,RegionName,State,City,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
0,112345.0,Maryvale,AZ,Phoenix,3.357290e+05,3.367033e+05,3.381803e+05,3.400057e+05,3.414316e+05,3.423949e+05,3.428074e+05,3.427356e+05,3.419887e+05,3.408405e+05,3.396659e+05,3.388768e+05
1,192689.0,Paradise,NV,Las Vegas,3.725479e+05,3.743522e+05,3.761585e+05,3.785524e+05,3.811490e+05,3.835971e+05,3.857815e+05,3.874718e+05,3.887102e+05,3.896048e+05,3.907356e+05,3.922168e+05
2,270958.0,Upper West Side,NY,New York,1.185179e+06,1.179215e+06,1.173972e+06,1.171712e+06,1.169479e+06,1.169561e+06,1.173572e+06,1.182601e+06,1.187323e+06,1.191276e+06,1.203791e+06,1.222023e+06
3,270957.0,Upper East Side,NY,New York,1.176590e+06,1.170734e+06,1.166905e+06,1.164981e+06,1.161944e+06,1.160380e+06,1.162163e+06,1.168415e+06,1.171441e+06,1.173235e+06,1.184594e+06,1.201830e+06
4,118208.0,South Los Angeles,CA,Los Angeles,6.790626e+05,6.726087e+05,6.658937e+05,6.633438e+05,6.633109e+05,6.635775e+05,6.661953e+05,6.714373e+05,6.780470e+05,6.834528e+05,6.871959e+05,6.899128e+05


In [64]:
# Pull only the data for Denver
denver_real_estate_neighborhood_df = real_estate_neighborhood_df.query("City == 'Denver'")
denver_real_estate_neighborhood_df.head()

,RegionID,RegionName,State,City,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
235,273809.0,Gateway - Green Valley Ranch,CO,Denver,491007.161995,491035.364888,491392.950975,491882.612105,491760.579762,490858.062361,489745.958237,488726.750030,487995.892604,487302.918410,486903.731696,486630.545212
329,6018.0,Montbello,CO,Denver,438542.958204,438864.819361,439648.745422,440609.850528,441051.882758,440704.584839,439855.248083,439006.425223,438219.703796,437274.333697,436726.770604,436244.288979
410,275564.0,Central Park,CO,Denver,811468.548208,808313.788628,807909.760277,809607.440587,810438.023097,808244.803889,805155.863258,802598.736735,801195.640708,799657.640124,798600.457230,798423.875011
602,268671.0,Hampden,CO,Denver,526324.587562,525805.315859,525939.414980,525984.010962,525244.032565,523673.717091,522134.523951,521460.718137,521413.062907,521744.191939,522074.783631,522563.712560
681,268662.0,Five Points,CO,Denver,620700.178593,619673.583635,619819.928185,620063.873246,618676.421526,615327.394116,611031.871498,607721.730126,605831.104045,604129.703444,603280.732065,603091.967420


In [65]:
# Check for null values
print(denver_real_estate_neighborhood_df.isnull().sum().compute())

RegionID      0
RegionName    0
State         0
City          0
2024-01-31    0
2024-02-29    0
2024-03-31    0
2024-04-30    0
2024-05-31    0
2024-06-30    0
2024-07-31    0
2024-08-31    0
2024-09-30    0
2024-10-31    0
2024-11-30    0
2024-12-31    0
dtype: int64


In [66]:
# Check datatypes
print(denver_real_estate_neighborhood_df.dtypes)

RegionID              float64
RegionName    string[pyarrow]
State         string[pyarrow]
City          string[pyarrow]
2024-01-31            float64
2024-02-29            float64
2024-03-31            float64
2024-04-30            float64
2024-05-31            float64
2024-06-30            float64
2024-07-31            float64
2024-08-31            float64
2024-09-30            float64
2024-10-31            float64
2024-11-30            float64
2024-12-31            float64
dtype: object


In [67]:
# Change column names 
denver_real_estate_neighborhood_df = denver_real_estate_neighborhood_df.rename(columns={'RegionName': 'Neighborhood'})
denver_real_estate_neighborhood_df.head()

,RegionID,Neighborhood,State,City,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
235,273809.0,Gateway - Green Valley Ranch,CO,Denver,491007.161995,491035.364888,491392.950975,491882.612105,491760.579762,490858.062361,489745.958237,488726.750030,487995.892604,487302.918410,486903.731696,486630.545212
329,6018.0,Montbello,CO,Denver,438542.958204,438864.819361,439648.745422,440609.850528,441051.882758,440704.584839,439855.248083,439006.425223,438219.703796,437274.333697,436726.770604,436244.288979
410,275564.0,Central Park,CO,Denver,811468.548208,808313.788628,807909.760277,809607.440587,810438.023097,808244.803889,805155.863258,802598.736735,801195.640708,799657.640124,798600.457230,798423.875011
602,268671.0,Hampden,CO,Denver,526324.587562,525805.315859,525939.414980,525984.010962,525244.032565,523673.717091,522134.523951,521460.718137,521413.062907,521744.191939,522074.783631,522563.712560
681,268662.0,Five Points,CO,Denver,620700.178593,619673.583635,619819.928185,620063.873246,618676.421526,615327.394116,611031.871498,607721.730126,605831.104045,604129.703444,603280.732065,603091.967420


In [68]:
#Find unique neighborhood names in real estate data
print(denver_real_estate_neighborhood_df['Neighborhood'].unique().compute())

0     Gateway - Green Valley Ranch
1                        Montbello
2                     Central Park
3                          Hampden
4                      Five Points
                  ...             
71                    Country Club
72                    Civic Center
73                        Rosedale
74    Denver International Airport
75                         Auraria
Name: Neighborhood, Length: 76, dtype: object


In [69]:
#Find unique neighborhood names in listings data
print(listings_df_cleaned['Neighborhood'].unique().compute())

0               Highland
1            Five Points
2        North Park Hill
3                  Baker
4     North Capitol Hill
             ...        
72           Harvey Park
73              Westwood
74     Harvey Park South
75        Elyria Swansea
76               Kennedy
Name: Neighborhood, Length: 77, dtype: object


In [75]:
# Find the average home price for each neighborhood in 2024 
# Example to get the average of 'Column1', 'Column2', and 'Column3' for each row
months_2024 = ['2024-01-31', '2024-02-29', '2024-03-31','2024-04-30', '2024-05-31', '2024-06-30', '2024-07-31', '2024-08-31', '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31']
denver_real_estate_neighborhood_df['Average Home Price 2024'] = real_estate_neighborhood_df[months_2024].mean(axis=1).round(2).compute()
denver_real_estate_neighborhood_df.head()

,RegionID,Neighborhood,State,City,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,Average Home Price 2024
235,273809.0,Gateway - Green Valley Ranch,CO,Denver,491007.161995,491035.364888,491392.950975,491882.612105,491760.579762,490858.062361,489745.958237,488726.750030,487995.892604,487302.918410,486903.731696,486630.545212,489603.54
329,6018.0,Montbello,CO,Denver,438542.958204,438864.819361,439648.745422,440609.850528,441051.882758,440704.584839,439855.248083,439006.425223,438219.703796,437274.333697,436726.770604,436244.288979,438895.80
410,275564.0,Central Park,CO,Denver,811468.548208,808313.788628,807909.760277,809607.440587,810438.023097,808244.803889,805155.863258,802598.736735,801195.640708,799657.640124,798600.457230,798423.875011,805134.55
602,268671.0,Hampden,CO,Denver,526324.587562,525805.315859,525939.414980,525984.010962,525244.032565,523673.717091,522134.523951,521460.718137,521413.062907,521744.191939,522074.783631,522563.712560,523696.84
681,268662.0,Five Points,CO,Denver,620700.178593,619673.583635,619819.928185,620063.873246,618676.421526,615327.394116,611031.871498,607721.730126,605831.104045,604129.703444,603280.732065,603091.967420,612445.71


In [ ]:
# Create a new dataframe without the monthly price data
cleaned_denver_real_estate_df = denver_real_estate_neighborhood_df[["RegionID", "Neighborhood", "State", "City", "Average Home Price 2024"]].copy()
cleaned_denver_real_estate_df.head()

,RegionID,Neighborhood,State,City,Average Home Price 2024
235,273809.0,Gateway - Green Valley Ranch,CO,Denver,489603.54
329,6018.0,Montbello,CO,Denver,438895.80
410,275564.0,Central Park,CO,Denver,805134.55
602,268671.0,Hampden,CO,Denver,523696.84
681,268662.0,Five Points,CO,Denver,612445.71


In [77]:
# Export the cleaned dataframe to a csv file
cleaned_denver_real_estate_df.to_csv('Resources/cleaned_denver_real_estate.csv', index=False, single_file=True)

['/Users/angelinamurdock/Desktop/Classwork/Project-5_Group-3/Resources/cleaned_denver_real_estate.csv']